<a href="https://colab.research.google.com/github/saeed-saffari/LLM_education_topic_modeling/blob/main/topic_modeling_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

### import libraries

In [9]:
#pip install pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer
import pyLDAvis.gensim as gensimvis
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pyLDAvis
pyLDAvis.enable_notebook()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### Loading data

In [14]:
df = pd.read_csv('https://github.com/saeed-saffari/LLM_education_topic_modeling/raw/refs/heads/main/Tastic%20AI%20-%20Opensource%20-%20AI%20tools.csv')
print('Shape:', df.shape)
print(df.columns)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Shape: (4665, 7)
Index(['title', 'minidesc', 'desc', 'cat', 'new_image', 'url-href', 'vote'], dtype='object')


,title,minidesc,desc,cat,new_image,url-href,vote
0,ChatGPT,ChatGPT optimizes language models for dialogue,ChatGPT FeaturesChatGPT is an AI-powered conve...,AI Productivity tools,https://tasticai.com/wp-content/uploads/aitool...,https://chat.openai.com/chat,4813.0
1,Namelix,"Generate short, brandable business names & dom...",Namelix FeaturesNamelix is an AI-powered busin...,AI startup tools,https://tasticai.com/wp-content/uploads/aitool...,https://namelix.com,4012.0
2,Tome,Unlock your best work with AI-powered generati...,Tome FeaturesThe future of generative storytel...,AI presentation tools,https://tasticai.com/wp-content/uploads/aitool...,https://tome.app,3593.0
3,Fliki,Create videos from blog posts in 2 mins,Fliki FeaturesFliki is an AI-powered text-to-v...,AI video Generator tools,https://tasticai.com/wp-content/uploads/aitool...,https://fliki.ai,3580.0
4,Midjourney,AI lab exploring new mediums to expand human i...,Midjourney FeaturesMidjourney is an independen...,AI tools for art,https://tasticai.com/wp-content/uploads/aitool...,https://www.midjourney.com/home,3000.0


In [16]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### clean data

In [18]:
def preprocess(text):
    if isinstance(text, str):
        tokens = word_tokenize(text.lower())
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
        return tokens
    return []

descriptions = df['desc'].tolist()
processed_descriptions = [preprocess(desc) for desc in descriptions]

# Create a dictionary from the processed descriptions
dictionary = corpora.Dictionary(processed_descriptions)

# Create a corpus (bag of words for each document)
corpus = [dictionary.doc2bow(text) for text in processed_descriptions]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Training model

In [19]:
# Train the LDA model
lda_model = LdaModel(corpus, num_topics=7, id2word=dictionary, passes=30) # passes: number of whole text reviews / num_topics: number of topics

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# Print the topics found by the LDA model
topics = lda_model.print_topics(num_words=10)
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

Topic 0: 0.031*"photo" + 0.023*"ai" + 0.022*"image" + 0.016*"user" + 0.015*"avatar" + 0.010*"style" + 0.010*"feature" + 0.009*"profile" + 0.009*"background" + 0.008*"tool"
Topic 1: 0.064*"content" + 0.016*"writing" + 0.014*"tool" + 0.013*"generate" + 0.013*"ai" + 0.012*"medium" + 0.012*"feature" + 0.011*"creation" + 0.011*"create" + 0.011*"user"
Topic 2: 0.023*"user" + 0.021*"personalized" + 0.011*"ai" + 0.009*"feature" + 0.009*"offer" + 0.008*"individual" + 0.008*"tool" + 0.008*"based" + 0.007*"case" + 0.007*"recommendation"
Topic 3: 0.031*"user" + 0.015*"ai" + 0.013*"tool" + 0.013*"language" + 0.013*"feature" + 0.009*"conversation" + 0.009*"information" + 0.007*"text" + 0.007*"email" + 0.007*"video"
Topic 4: 0.023*"customer" + 0.021*"ai" + 0.016*"business" + 0.013*"user" + 0.012*"support" + 0.010*"tool" + 0.009*"feature" + 0.009*"data" + 0.008*"product" + 0.007*"search"
Topic 5: 0.027*"ai" + 0.022*"image" + 0.021*"user" + 0.018*"create" + 0.014*"tool" + 0.012*"feature" + 0.012*"desig

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# Prepare the visualization
lda_display = gensimvis.prepare(lda_model, corpus, dictionary)

# Show the visualization in a browser
pyLDAvis.display(lda_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
